In [ ]:
import pickle
import nltk
from collections import defaultdict
import random
import numpy as np
from tqdm.notebook import tqdm
import re

In [ ]:
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
print(os.getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [ ]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Dataset")
os.getcwd() 

'/content/gdrive/My Drive/Colab Notebooks/Dataset'

In [ ]:
with open("Preprocessed Data/Twitter.txt", "r", encoding = "utf-8") as f:
    data = f.read()
    
words = data.split()    
print("Number of words:", len(words))

Number of words: 14014421


In [ ]:
def split_to_sentences(data, point):
    """
    Split data by linebreak point(can be '\n')
    
    Args:
        data: str
    
    Returns:
        A list of sentences
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    sentences = data.split(point)
    ### END CODE HERE ###
    
    # Additional clearning (This part is already implemented)
    # - Remove leading and trailing spaces from each sentence
    # - Drop sentences if they are empty strings.
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    
    return sentences 

In [ ]:
new_data = split_to_sentences(data, '.')
new_data[53]

'mmmugh'

In [ ]:
sentences_words = []
for i in new_data:
  temp = i.split()
  sentences_words.append(temp)

random.shuffle(sentences_words)
sentences_words[:2]

[['a', 'lottle', 'bit'],
 ['we',
  'know',
  'how',
  'to',
  'destroy',
  'planet',
  'reject',
  'the',
  'recession',
  'rofl',
  'youtube',
  'it',
  'make',
  'sme',
  'lol',
  'am',
  'church',
  'service']]

In [ ]:
print(len(sentences_words))

824700


In [ ]:
random.shuffle(sentences_words)

new_data1 = sentences_words[:int(len(sentences_words)/56)] # 16m to 250k words downgrade. - 16m has 798k lines  63/2
print(len(new_data1))

14726


In [ ]:
test_data = sentences_words[int(len(sentences_words)/56): int(len(sentences_words)/56) + 2000]

In [ ]:
# for Movies - 63
# for News - 67
# for Twitter - 56
# for Therapy - 4.3

In [ ]:
new_data1[0]

['hug', 'it', 'time', 'for', 'a', 'new', 'blackberry', 'battery']

In [ ]:
# model takes sentences not tokens    
def Building(new_data):
  sentences = []
  pre_data = ""
  for i in tqdm(new_data):
    temp = ' '.join([t for t in i if t])
    sentences.append(temp)
    
  for p in tqdm(sentences):
    temp = ''.join([t for t in p if t])
    # print(temp)
    pre_data = pre_data + temp + " "

  return pre_data

pre_data = Building(new_data1)
print(pre_data[:30])


hug it time for a new blackber


In [ ]:
pre_test_data = Building(test_data)
print(pre_test_data[:30])


he is developer to fm integrat


In [ ]:
fp = pre_data.split()
word_count = len(fp)
print(word_count)

250807


In [ ]:
def create_seq(text, lengths):
    length = lengths # changed from 30 to 15
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(pre_data, 10)

Total Sequences: 1250859


In [ ]:
sequences_test = create_seq(pre_test_data, 10)

Total Sequences: 166607


In [ ]:
# create a character mapping index
chars = sorted(list(set(pre_data)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [ ]:
sequences_test = encode_seq(sequences_test)

In [ ]:
from sklearn.model_selection import train_test_split


# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (1000687, 10) Val shape: (250172, 10)


In [ ]:
# For making test data
sequences_test = np.array(sequences_test)
X_test, y_test = sequences_test[:,:-1], sequences_test[:,-1]
# one hot encode y
y_test = to_categorical(y_test, num_classes=vocab)
X_ttr, X_tval, y_ttr, y_tval = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

print('Train shape:', X_ttr.shape, 'Val shape:', X_tval.shape)

Train shape: (133285, 10) Val shape: (33322, 10)


In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=10)) #input_length=10 as characters we took were 10.
model.add(GRU(100, return_sequences = True)) # dropped from 150 to 100
model.add(GRU(100))
# model.add(Dense(100, activation = 'relu'))
model.add(Dense(vocab, activation = 'softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=15, validation_data=(X_val, y_val), batch_size=512)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 10, 50)            1350      
_________________________________________________________________
gru_18 (GRU)                 (None, 10, 100)           45600     
_________________________________________________________________
gru_19 (GRU)                 (None, 100)               60600     
_________________________________________________________________
dense_13 (Dense)             (None, 27)                2727      
Total params: 110,277
Trainable params: 110,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15
2138/2138 [==============================] - 46s 20ms/step - loss: 2.2298 - acc: 0.3429 - val_loss: 1.6557 - val_acc: 0.4956
Epoch 2/15
2138/2138 [==============================] - 42s 20ms/step - loss: 1.6096 - acc: 0.5092 - val

In [ ]:
# saving the model
# serialize to JSON
json_file = model.to_json()
with open("Model-JSON/Twitter_character.json", "w") as file:
   file.write(json_file)

# serialize weights to HDF5
model.save_weights("Model-JSON/Twitter-weights-character.h5")

In [ ]:
from keras.models import model_from_json
# load json and create model
file = open(r'Model-JSON/Therapy_character.json', 'r')
model_json = file.read()
file.close()
loaded_model = model_from_json(model_json)
# load weights
loaded_model.load_weights(r'Model-JSON/Therapy-weights-character.h5')

print()

In [ ]:
results = model.evaluate(X_ttr, y_ttr)
print('Test accuracy: {0:.2f}%'.format(results[1]*100))

5491/5491 [==============================] - 28s 5ms/step - loss: 1.3639 - acc: 0.5793
Test accuracy: 57.93%


In [ ]:
loaded_model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')


results_cross = loaded_model.evaluate(X_ttr, y_ttr)
print('Test accuracy: {0:.2f}%'.format(results_cross[1]*100))

4166/4166 [==============================] - 14s 3ms/step - loss: 2.1472 - acc: 0.4602
Test accuracy: 46.15%


In [ ]:
results
# Test loss       Test accuracy